In [23]:
from dplython import (DplyFrame, X, diamonds, select, sift, sample_n
                      , sample_frac, head, arrange, mutate, group_by
                      , summarize, DelayFunction)
import pandas as pd
import numpy as np

In [5]:
'''
uses piping. 
'''

path = '/Users/tlee010/desktop/dsi-sf-2-timdavidlee/datasets/football_combine/combine.csv'

In [7]:
combine = DplyFrame(pd.read_csv(path))
combine.head(2)

/Users/tlee010/anaconda/envs/dsi/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/tlee010/anaconda/envs/dsi/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/Users/tlee010/anaconda/envs/dsi/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/Users/tlee010/anaconda/envs/dsi/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/Users/tlee010/ana

,id,Year,Name,Position,HeightFeet,HeightInches,Weight,Arms,Hands,FortyYD,...,Bench,Round,College,Pick,PickRound,PickTotal,FirstName,LastName,HeightInchesTotal,Wonderlic
0,8984,2013,Quanterus Smith,DE,6,5.0,250,33.25,10.375,0.0,...,0,5,Western Kentucky,13(146),13,146,Quanterus,Smith,77.0,0
1,9002,2013,Abry Jones,DT,6,3.0,313,35.00,9.750,0.0,...,30,0,NaN,NaN,0,0,Abry,Jones,75.0,0


In [8]:
#position = combine['Position']
position = combine >> select(X.Position)
print position[0:5]

  Position
0       DE
1       DT
2       DT
3       FS
4       FS


In [9]:
players = combine[['Name','Weight']]

In [10]:
players = players[players.Weight < 200]
players.head()

,Name,Weight
15,Brandon Boykin,182
49,Foswhitt Whittaker,193
55,Ryan Broyles,192
56,Chase Minnifield,183
58,Shaun Prater,190


In [11]:
dply_players = (combine >> sift(X.Weight <200) >> select(X.Name, X.Weight))
dply_players.head()

,Name,Weight
15,Brandon Boykin,182
49,Foswhitt Whittaker,193
55,Ryan Broyles,192
56,Chase Minnifield,183
58,Shaun Prater,190


In [19]:
# PANDAS
# Take a random sample of 5% and take out the Name and Weight 
sample = combine[["Name","Weight"]]
sample = sample.sample(frac=.05)
sample.head()

,Name,Weight
1254,Larry Johnson,228
932,Curtis Brown,185
3201,Kenny Iwebema,274
170,Michael Smith,207
3911,Hebron Fangupo,323


In [20]:
# DPLYTHON

dply_sample = (combine >> sample_frac(0.05)>> select (X.Name, X.Weight))
dply_sample.head(2)

,Name,Weight
4055,Jeff Smith,286
111,Cairo Santos,164


In [25]:
# Group by postion year and weight and marms
# return the mean of that column

def weight_plus_arms_mean(df):
    summed = df['Weight'] + df['Arms']
    df['weight_arms_mean'] = np.mean(summed)
    return df.loc[:,'weight_arms_mean'].iloc[0]

combine.groupby(['Position','Year']).apply(weight_plus_arms_mean).reset_index().head()


,Position,Year,0
0,C,2014,307.166667
1,CB,1999,184.807692
2,CB,2000,191.750000
3,CB,2001,193.655172
4,CB,2002,194.333333


In [29]:
# dplyr version:
wa_mean = (combine >> 
                    group_by(X.Position, X.Year) >> 
                    mutate(weight_arms_sum=X.Weight + X.Arms) >>
                    summarize(weight_arms_mean=X.weight_arms_sum.mean())
                   )
wa_mean.head(3)

,Position,Year,weight_arms_mean
0,C,2014,307.166667
1,CB,1999,184.807692
2,CB,2000,191.750000


In [31]:
@DelayFunction
def mean_of_columns(df):
    meancol = df.mean(axis=1)
    return meancol

# mean of year weight and hands columns:
ywh_mean = (combine >>
            select(X.Name, X.Weight, X.Year, X.Hands) >>
            mutate(mean_of_ywh = (combine >>
                                 select (X.Year, X.Weight, X.Hands)>>
                                 mean_of_columns(X)
                                 )
                  )
           )

ywh_mean.head()

,Name,Weight,Year,Hands,mean_of_ywh
0,Quanterus Smith,250,2013,10.375,757.791667
1,Abry Jones,313,2013,9.750,778.583333
2,Bennie Logan,309,2013,10.250,777.416667
3,John Boyett,204,2013,8.500,741.833333
4,Bacarri Rambo,211,2013,9.250,744.416667
